# Question 2


In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# Loads data.
#dataset = spark.read.format("libsvm").load("/Users/gongqian/spark-2.2.0-bin-hadoop2.7/data/mllib/sample_kmeans_data.txt")

#dataset from Reuters datasets
dataset = spark.read.text("/Users/littleostrichsnewmacbook/Desktop/bigdata-hw2/Q2/newsCorpora.csv").rdd 
parts = dataset.map(lambda row: row.value.split("	"))
rating = parts.map(lambda p: Row(ID=(p[0]),TITLE=(p[1]),URL=(p[2]),PUBLISHER=(p[3]),CATEGORY=(p[4]),STORY=(p[5]),HOSTNAME=(p[6]),TIMESTAMP=(p[7])))
#rating = parts.map(lambda p: Row(label = (p[0]),features = Vectors.sparse([p[1], p[2], p[3], p[4], p[5], p[6], p[7]])))
ratings = spark.createDataFrame(rating)
ratings.show(5,False)

+--------+-------------------+---+-----------------+-----------------------------+-------------+---------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+
|CATEGORY|HOSTNAME           |ID |PUBLISHER        |STORY                        |TIMESTAMP    |TITLE                                                                |URL                                                                                                                        |
+--------+-------------------+---+-----------------+-----------------------------+-------------+---------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+
|b       |www.latimes.com    |1  |Los Angeles Times|ddUyU0VZz0BRneMioxUPQVP6sIxvM|1394470370698|Fed official says weak data cau

In [2]:
tokenizer = Tokenizer(inputCol="TITLE", outputCol="words")
wordsData = tokenizer.transform(ratings)
wordsData.select("words").show(5,False)

hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=10)
featurizedData = hashingTF.transform(wordsData)
featurizedData.select("features").show(5,False)

+---------------------------------------------------------------------------------+
|words                                                                            |
+---------------------------------------------------------------------------------+
|[fed, official, says, weak, data, caused, by, weather,, should, not, slow, taper]|
|[fed's, charles, plosser, sees, high, bar, for, change, in, pace, of, tapering]  |
|[us, open:, stocks, fall, after, fed, official, hints, at, accelerated, tapering]|
|[fed, risks, falling, 'behind, the, curve',, charles, plosser, says]             |
|[fed's, plosser:, nasty, weather, has, curbed, job, growth]                      |
+---------------------------------------------------------------------------------+
only showing top 5 rows

+--------------------------------------------------+
|features                                          |
+--------------------------------------------------+
|(10,[0,3,5,6,8],[1.0,2.0,4.0,1.0,4.0])            |
|(10,[3

In [4]:
# Trains a k-means model.
kmeans = KMeans().setK(100).setSeed(1)
model = kmeans.fit(featurizedData)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(featurizedData)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 1547146.40047
Cluster Centers: 
[ 1.78315946  0.76154993  2.62816692  3.75298063  0.50931446  1.0257079
  0.79210134  0.80216095  0.67622951  0.57600596]
[ 0.49172088  1.35215849  0.39178001  0.56667652  0.44869899  0.43583678
  0.52335896  0.          0.55322295  0.45609107]
[ 2.37891953  0.37797507  1.92179826  0.45202116  0.39516434  0.50358897
  0.30487344  0.5         0.67434832  0.39667548]
[ 0.65325474  0.29830856  0.59251666  0.55279344  0.37314198  2.13864685
  1.88031779  0.54587391  2.48795489  0.48487955]
[ 0.57016408  0.51791883  0.44926598  2.44019862  2.37262522  0.58160622
  0.60729706  0.50949914  0.92227979  0.4529361 ]
[ 0.44388398  0.54930643  0.65750315  0.67692308  2.35989912  0.40504414
  2.11576293  0.55460277  0.41008827  0.56620429]
[ 1.90647021  1.81806534  0.68225496  0.45515695  0.53299167  1.2309417
  0.61819347  2.28090967  0.53907751  0.67552851]
[ 1.74569402  1.33130699  2.59270517  0.74265451  0.65214455  0.63053023
 